In [40]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import datacompy
import os
import numpy as np

# narzedzia
from sklearn.model_selection import (
    train_test_split,
    cross_val_score,
    cross_val_predict,
    learning_curve,
    RepeatedStratifiedKFold,
    GridSearchCV
)
from collections import Counter
from sklearn.metrics import classification_report, confusion_matrix
from sklearn import tree
from sklearn.tree import plot_tree
from sklearn.preprocessing import LabelEncoder

# modele + Smote
from xgboost import XGBClassifier, XGBRFClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

from imblearn.over_sampling import SMOTE, BorderlineSMOTE
from ctgan import CTGAN

num_duplicates = df.duplicated().sum()
print(f"Liczba duplikatów: {num_duplicates}")

### Data PreProcessing

In [42]:
df = pd.read_csv("D:/ml/xgboost-main/data/RT-IoT/RT_IOT2022")
df = df.drop(columns =["Unnamed: 0"])
#calasification: if normal communication: 1, attacks: 0
df['label'] = df['Attack_type'].isin(['MQTT_Publish', 'Thing_Speak', 'Wipro_bulb']).astype(int)
df = df.drop_duplicates()
#drop from column="proto" when values="icmp" & tcp=1 udp=0
df = df[df['proto'] != 'icmp']
label = {'tcp':1,'udp':0}      
df = df.replace(label)
#change text from column="service" to int values by LabelEncoder()
le = LabelEncoder()
df['service_en'] = le.fit_transform(df['service'])
cols = df.columns.tolist()
index = cols.index('service')                                                   #find column='service'
cols.insert(index + 1, cols.pop(cols.index('service_en')))                      #paste column='service_encoded' after column='service'
df = df[cols]
mapping = dict(zip(le.classes_, le.transform(le.classes_)))                     #mapping text = value
print("\nMapped values for 'service_en':")
for name, encoded_value in mapping.items():
    print(f"'{name}' -> {encoded_value}")
df = df.drop(columns=["service"])                                               #drop column="service"
#change text from column="Attack_type" to int values by LabelEncoder()
df['Attack_type_en'] = le.fit_transform(df['Attack_type'])
cols = df.columns.tolist()
index = cols.index('Attack_type')                                               #find column='Attack_type'
cols.insert(index + 1, cols.pop(cols.index('Attack_type_en')))                  #paste column='Attack_type_en' after column='Attack_type'
df = df[cols]
mapping = dict(zip(le.classes_, le.transform(le.classes_)))                     #mapping text = value
print("\nMapped values for 'Attack_type_en':")
for name, encoded_value in mapping.items():
    print(f"'{name}' -> {encoded_value}")
df = df.drop(columns=["Attack_type"])                                           #drop column="Attack_tyope"
df.dtypes
df.head()


Mapped values for 'service_en':
'-' -> 0
'dhcp' -> 1
'dns' -> 2
'http' -> 3
'irc' -> 4
'mqtt' -> 5
'ntp' -> 6
'radius' -> 7
'ssh' -> 8
'ssl' -> 9

Mapped values for 'Attack_type_en':
'ARP_poisioning' -> 0
'DDOS_Slowloris' -> 1
'DOS_SYN_Hping' -> 2
'MQTT_Publish' -> 3
'Metasploit_Brute_Force_SSH' -> 4
'NMAP_FIN_SCAN' -> 5
'NMAP_OS_DETECTION' -> 6
'NMAP_TCP_scan' -> 7
'NMAP_UDP_SCAN' -> 8
'NMAP_XMAS_TREE_SCAN' -> 9
'Thing_Speak' -> 10
'Wipro_bulb' -> 11


,id.orig_p,id.resp_p,proto,service_en,flow_duration,fwd_pkts_tot,bwd_pkts_tot,fwd_data_pkts_tot,bwd_data_pkts_tot,fwd_pkts_per_sec,...,idle.min,idle.max,idle.tot,idle.avg,idle.std,fwd_init_window_size,bwd_init_window_size,fwd_last_window_size,Attack_type_en,label
0,38667,1883,1,5,32.011598,9,5,3,3,0.281148,...,2.972918e+07,2.972918e+07,2.972918e+07,2.972918e+07,0.0,64240,26847,502,3,1
1,51143,1883,1,5,31.883584,9,5,3,3,0.282277,...,2.985528e+07,2.985528e+07,2.985528e+07,2.985528e+07,0.0,64240,26847,502,3,1
2,44761,1883,1,5,32.124053,9,5,3,3,0.280164,...,2.984215e+07,2.984215e+07,2.984215e+07,2.984215e+07,0.0,64240,26847,502,3,1
3,60893,1883,1,5,31.961063,9,5,3,3,0.281593,...,2.991377e+07,2.991377e+07,2.991377e+07,2.991377e+07,0.0,64240,26847,502,3,1
4,51087,1883,1,5,31.902362,9,5,3,3,0.282111,...,2.981470e+07,2.981470e+07,2.981470e+07,2.981470e+07,0.0,64240,26847,502,3,1
